In [156]:
import pandas as pd
import pyodbc

In [198]:
restaurante_path = "Restaurantes_base.csv"
hobbies_path = "Sports_and_Outdoors_5.csv"
steam_reviews_path = "steam_reviews.csv"
nba_data_path1 = "game.csv"
nba_data_path2 = "inactive_players.csv"
conciertos_path = "final_combined_dataframe.csv"

In [199]:
def read_csv(df_path):
    try:
        data = pd.read_csv(df_path, encoding='latin1', on_bad_lines='skip')
        return data
    except FileNotFoundError:
        print("Archivo no encontrado")
        return None
    except Exception as e:
        print(f"Error al leer archivo: {e}")
        return None

In [200]:
def read_json(path):
    try:
        data = pd.read_json(path, lines=False)
        return data
    except FileNotFoundError:
        print(f"File {path} not found")
    except ValueError as e:
        print(f"ValueError: {e}")

In [208]:
df_restaurante = read_csv(restaurante_path)
print(len(df_restaurante))

202063


C:\Users\Keepc\AppData\Local\Temp\ipykernel_5984\2988551617.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(df_path, encoding='latin1', on_bad_lines='skip')


In [209]:
df_hobbies = read_csv(hobbies_path)
print(len(df_hobbies))

286557


In [210]:
df_steam = read_csv(steam_reviews_path)
print(len(df_steam))

203111


In [211]:
df_nba1 = read_csv(nba_data_path1)
df_nba2 = read_csv(nba_data_path2)

df_nba = pd.concat([df_nba1, df_nba2], axis=0, ignore_index=True)
print(len(df_nba))

175889


In [212]:
df_conciertos = read_csv(conciertos_path)
print(len(df_conciertos))

C:\Users\Keepc\AppData\Local\Temp\ipykernel_5984\2988551617.py:3: DtypeWarning: Columns (16,17,18,48,49,50,64,65,66,80,81,82,84,85,126,130,131,132,133,134,137,140) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(df_path, encoding='latin1', on_bad_lines='skip')


295941


In [213]:
conexion_str = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=ADRIAN\\MSSQLSERVER01;DATABASE=datasets_analisis;Trusted_Connection=yes;"
conexion = pyodbc.connect(conexion_str)

In [214]:
def guardar_sqlserver(df, nombre_tabla):

    cursor = conexion.cursor()
    columnas = ", ".join([f"[{col}] NVARCHAR(MAX)" for col in df.columns])
    create_table_query = f"IF OBJECT_ID('{nombre_tabla}', 'U') IS NULL CREATE TABLE {nombre_tabla} ({columnas})"
    
    cursor.execute(create_table_query)
    
    conexion.commit()
    
    for _, row in df.iterrows():
        valores = "', '".join(str(x).replace("'", "''") for x in row.values)
        insert_query = f"INSERT INTO {nombre_tabla} VALUES ('{valores}')"

        cursor.execute(insert_query)
        
    conexion.commit()
    print(f"Datos guardados en la tabla {nombre_tabla}")
    
    cursor.close()  

In [215]:
guardar_sqlserver(df_nba, "datos_nba")

Datos guardados en la tabla datos_nba


In [216]:
guardar_sqlserver(df_restaurante, "datos_restaurantes")

Datos guardados en la tabla datos_restaurantes


In [217]:
guardar_sqlserver(df_hobbies, "datos_hobbies")

Datos guardados en la tabla datos_hobbies


In [228]:
# Normalizar nombres de columnas
df_conciertos.columns = df_conciertos.columns.str.strip().str.lower()

# Eliminar columnas duplicadas completamente (con nombres normalizados)
df_conciertos = df_conciertos.loc[:, ~df_conciertos.columns.duplicated()]

print(df_conciertos.columns)


Index(['_id', 'unnamed: 0', 'game_id', 'period_id', 'time_seconds', 'team_id',
       'player_id', 'start_x', 'start_y', 'end_x',
       ...
       'weight', 'currentteamid', 'birthdate', 'height', 'role', 'birtharea',
       'wyid', 'foot', 'shortname', 'currentnationalteamid'],
      dtype='object', length=153)


In [229]:
guardar_sqlserver(df_conciertos, "datos_conciertos")

Datos guardados en la tabla datos_conciertos


In [219]:
guardar_sqlserver(df_steam, "datos_steam")

Datos guardados en la tabla datos_steam


In [232]:
df_unificado = pd.concat([df_restaurante, df_nba, df_hobbies, df_conciertos, df_steam], axis=0, ignore_index=True)
print(f"Total datos NBA: {len(df_nba)}")
print(f"Total datos restaurantes: {len(df_restaurante)}")
print(f"Total datos hobbies: {len(df_hobbies)}")
print(f"Total datos steam: {len(df_steam)}")
print(f"Total datos steam: {len(df_conciertos)}")
print(f"Total datos: {len(df_unificado)}")

Total datos NBA: 175889
Total datos restaurantes: 202063
Total datos hobbies: 286557
Total datos steam: 203111
Total datos steam: 295941
Total datos: 1163561


In [233]:
df_unificado.tail(100)

,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,...,shortname,currentnationalteamid,app_id,app_name,review_text,review_score,review_votes,source_collection,Polaridad,Sentimiento
1163461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,550.0,Left 4 Dead 2,"When I buy a game, I expect it to f**cking wor...",-1.0,0.0,l4d2_reviews,-0.9030,Negativo
1163462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,550.0,Left 4 Dead 2,6/10 Gameplay is fantastic whereas servers su...,-1.0,0.0,l4d2_reviews,0.7367,Positivo
1163463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,550.0,Left 4 Dead 2,Nothing special just shooting and escaping.,-1.0,0.0,l4d2_reviews,-0.2635,Negativo
1163464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,550.0,Left 4 Dead 2,FREE TO PLAY LEFT 4 DEAD 2 CONGRATULATIONS VALVE!,-1.0,1.0,l4d2_reviews,0.6800,Positivo
1163465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,550.0,Left 4 Dead 2,This game got boring pretty fast. I would not ...,-1.0,0.0,l4d2_reviews,-0.0351,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,550.0,Left 4 Dead 2,Screensick (ï¼¾ï½ï¼¾),-1.0,0.0,l4d2_reviews,0.0000,Neutral
1163557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,550.0,Left 4 Dead 2,The online community is the most toxic I've ev...,-1.0,0.0,l4d2_reviews,0.1531,Positivo
1163558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,550.0,Left 4 Dead 2,"Somehow, this game doesn't do it for me. I've ...",-1.0,0.0,l4d2_reviews,0.0598,Positivo
1163559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,550.0,Left 4 Dead 2,"Lots of fun to play with friends, but the comm...",-1.0,0.0,l4d2_reviews,0.9811,Positivo


In [236]:
df_unificado.to_csv('mi_archivo.csv', index=False)